# Feature Engineering Notebook

## Overview
This notebook demonstrates all feature engineering techniques used in the property valuation pipeline.

**Learning Goals:**
1. Understand why certain features are engineered
2. Learn transformation techniques (log, ratios, interactions)
3. Implement domain-specific real estate features
4. Visualize the impact of engineered features

**Key Insight:** Feature engineering is about domain knowledge + creative thinking. We create features that capture business logic and relationships the model might not discover on its own.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Inspect Data
First, load the training data to begin feature engineering.

In [ ]:
# Load data
df = pd.read_excel('../data/raw/train.xlsx')
print(f"Data Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
display(df.head())

## 2. Temporal Features (Date Extraction)

### Why?
- **Real Estate Pattern:** Property prices vary by season. Summer sales may show different patterns than winter.
- **Trend Capture:** The year of sale captures market trends (prices tend to increase over time).
- **Seasonality:** Month reveals seasonal demand patterns.

### Feature List:
- `year`: Year of sale
- `month`: Month of sale (1-12)
- `season`: Categorical season (Winter=0, Spring=1, Summer=2, Fall=3)

### Example:
- **2014/05 → Year:2014, Month:5, Season:1 (Spring)**
- Spring properties may sell 2-5% higher due to market demand
- Year captures inflation and market appreciation

In [ ]:
# Create a sample date column for demonstration
df_temp = df.copy()

if 'date' in df_temp.columns:
    # Parse date
    df_temp['date'] = pd.to_datetime(df_temp['date'], format='%Y%m%dT%H%M%S')
    
    # Extract temporal features
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['month'] = df_temp['date'].dt.month
    
    # Map month to season
    def get_season(month):
        if month in [12, 1, 2]:
            return 0  # Winter
        elif month in [3, 4, 5]:
            return 1  # Spring
        elif month in [6, 7, 8]:
            return 2  # Summer
        else:
            return 3  # Fall
    
    df_temp['season'] = df_temp['month'].apply(get_season)
    df_temp = df_temp.drop('date', axis=1)
    
    print("✅ Temporal Features Created")
    print("\nSample:")
    display(df_temp[['year', 'month', 'season']].head())
    
    # Visualization: Average price by season
    season_names = {0: 'Winter', 1: 'Spring', 2: 'Summer', 3: 'Fall'}
    if 'price' in df_temp.columns:
        season_prices = df_temp.groupby('season')['price'].mean().sort_index()
        season_prices.index = [season_names[i] for i in season_prices.index]
        
        plt.figure(figsize=(10, 5))
        sns.barplot(x=season_prices.index, y=season_prices.values, palette='coolwarm')
        plt.title('Average Property Price by Season')
        plt.ylabel('Average Price ($)')
        plt.show()
else:
    print("⚠️ No 'date' column found. Skipping temporal features.")

## 3. Renovation Status Features

### Why?
- **Binary Indicator:** Whether a property was renovated significantly affects price.
- **Time Since Renovation:** Older renovations lose value over time.
- **Real Estate Logic:** A recently renovated home commands a premium; a renovation 20 years ago has depreciated.

### Feature List:
- `was_renovated`: Binary (0/1) - Has the property ever been renovated?
- `years_since_renovation`: Continuous - How long ago was the renovation?

### Example:
- **Property A:** Renovated in 2020 (sold in 2023) → `was_renovated=1`, `years_since_renovation=3`
- **Property B:** Never renovated (`yr_renovated=0`) → `was_renovated=0`, `years_since_renovation=0`
- Property A may sell for 15-25% premium despite Property B's higher original grade

In [ ]:
df_reno = df.copy()

if 'yr_renovated' in df_reno.columns and 'year' in df_reno.columns:
    # Binary renovation flag
    df_reno['was_renovated'] = (df_reno['yr_renovated'] > 0).astype(int)
    
    # Years since renovation
    df_reno['years_since_renovation'] = df_reno['year'] - df_reno['yr_renovated']
    df_reno.loc[df_reno['yr_renovated'] == 0, 'years_since_renovation'] = 0
    
    print("✅ Renovation Features Created")
    print(f"\nRenovated properties: {df_reno['was_renovated'].sum()} ({df_reno['was_renovated'].mean()*100:.1f}%)")
    print(f"Never renovated: {(df_reno['was_renovated'] == 0).sum()}")
    
    # Visualization
    if 'price' in df_reno.columns:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Price by renovation status
        reno_names = {0: 'Never Renovated', 1: 'Renovated'}
        sns.boxplot(data=df_reno, x='was_renovated', y='price', ax=axes[0], palette='Set2')
        axes[0].set_title('Price Distribution by Renovation Status')
        axes[0].set_xticklabels(['Never Renovated', 'Renovated'])
        axes[0].set_ylabel('Price ($)')
        
        # Years since renovation vs price (only renovated properties)
        df_renovated = df_reno[df_reno['was_renovated'] == 1]
        sns.scatterplot(data=df_renovated, x='years_since_renovation', y='price', ax=axes[1], alpha=0.5)
        axes[1].set_title('Years Since Renovation vs. Price')
        axes[1].set_xlabel('Years Since Renovation')
        axes[1].set_ylabel('Price ($)')
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Missing 'yr_renovated' or 'year' columns.")

## 4. Property Age

### Why?
- **Depreciation:** Older buildings typically worth less (unless historic/prime location).
- **Maintenance:** Older properties need more repairs and upgrades.
- **Building Codes:** Modern properties meet current safety/energy standards.

### Formula:
```
property_age = sale_year - yr_built
```

### Example:
- **House A:** Built in 2010, sold in 2023 → Age = 13 years
- **House B:** Built in 1995, sold in 2023 → Age = 28 years
- House A typically 20-30% more expensive (newer construction)

In [ ]:
df_age = df.copy()

if 'yr_built' in df_age.columns and 'year' in df_age.columns:
    df_age['property_age'] = df_age['year'] - df_age['yr_built']
    
    print("✅ Property Age Feature Created")
    print(f"\nProperty Age Statistics:")
    print(df_age['property_age'].describe())
    
    # Visualization
    if 'price' in df_age.columns:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Age distribution
        axes[0].hist(df_age['property_age'], bins=50, color='skyblue', edgecolor='black')
        axes[0].set_title('Distribution of Property Age')
        axes[0].set_xlabel('Age (years)')
        axes[0].set_ylabel('Count')
        
        # Age vs Price
        sns.scatterplot(data=df_age, x='property_age', y='price', ax=axes[1], alpha=0.5)
        axes[1].set_title('Property Age vs. Price')
        axes[1].set_xlabel('Age (years)')
        axes[1].set_ylabel('Price ($)')
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Missing 'yr_built' or 'year' columns.")

## 5. Space Ratio Features

### Why?
- **Living Space Efficiency:** How much of the lot is actually used for living?
- **Investment Potential:** High ratio = more building potential; low ratio = land premium.
- **Neighborhood Comparison:** Is this property above/below average for its area?

### Feature List:
- `sqft_per_lot`: Living area / Lot area (measures utilization)
- `above_neighborhood`: Binary - Is living sqft above neighborhood average?
- `total_rooms`: Bedrooms + Bathrooms

### Example:
- **Property A:** 3000 sqft living, 10,000 sqft lot → Ratio = 0.30 (dense use)
- **Property B:** 1500 sqft living, 10,000 sqft lot → Ratio = 0.15 (more land value)
- Ratios reveal building investment patterns

In [ ]:
df_ratio = df.copy()

# Total rooms
if 'bedrooms' in df_ratio.columns and 'bathrooms' in df_ratio.columns:
    df_ratio['total_rooms'] = df_ratio['bedrooms'] + df_ratio['bathrooms']
    df_ratio['bedrooms_per_bathroom'] = df_ratio['bedrooms'] / (df_ratio['bathrooms'] + 0.01)
    print("✅ Room Ratio Features Created")

# Sqft ratios
if 'sqft_living' in df_ratio.columns and 'sqft_lot' in df_ratio.columns:
    df_ratio['sqft_per_lot'] = df_ratio['sqft_living'] / (df_ratio['sqft_lot'] + 1)
    print("✅ Space Utilization Features Created")
    
    # Neighborhood comparison
    if 'sqft_living15' in df_ratio.columns:
        df_ratio['above_neighborhood'] = (df_ratio['sqft_living'] > df_ratio['sqft_living15']).astype(int)
        print("✅ Neighborhood Comparison Feature Created")

# Statistics
if 'sqft_per_lot' in df_ratio.columns:
    print(f"\nSpace Utilization Statistics:")
    print(df_ratio['sqft_per_lot'].describe())
    
    # Visualization
    if 'price' in df_ratio.columns:
        plt.figure(figsize=(12, 5))
        sns.scatterplot(data=df_ratio, x='sqft_per_lot', y='price', alpha=0.5)
        plt.title('Space Utilization Ratio vs. Price')
        plt.xlabel('sqft_living / sqft_lot')
        plt.ylabel('Price ($)')
        plt.show()

## 6. Quality Score (Multiplicative Interaction)

### Why?
- **Non-Linear Relationship:** Grade AND condition together matter more than sum.
- **Interaction Feature:** A house with grade=10 & condition=5 is worth more than grade=5 & condition=10.
- **Business Logic:** Premium features (high grade + high condition) create premium value.

### Features:
- `quality_score = grade × condition` (multiplicative)
- `quality_grade_condition = grade + condition` (additive alternative)

### Example:
- **Property A:** Grade=10, Condition=4 → Quality = 40 (good bones, needs work)
- **Property B:** Grade=8, Condition=5 → Quality = 40 (same score, different profile)
- **Property C:** Grade=9, Condition=5 → Quality = 45 (better overall)

Multiplicative captures the synergy: top-grade + top-condition commands exponential premium.

In [ ]:
df_quality = df.copy()

if 'grade' in df_quality.columns and 'condition' in df_quality.columns:
    # Multiplicative interaction
    df_quality['quality_score'] = df_quality['grade'] * df_quality['condition']
    # Additive alternative
    df_quality['quality_grade_condition'] = df_quality['grade'] + df_quality['condition']
    
    print("✅ Quality Score Features Created")
    print(f"\nQuality Score Statistics (Multiplicative):")
    print(df_quality['quality_score'].describe())
    
    # Correlation with price
    if 'price' in df_quality.columns:
        corr_mult = df_quality['quality_score'].corr(df_quality['price'])
        corr_add = df_quality['quality_grade_condition'].corr(df_quality['price'])
        print(f"\nCorrelation with Price:")
        print(f"  Multiplicative (grade × condition): {corr_mult:.4f}")
        print(f"  Additive (grade + condition): {corr_add:.4f}")
        print(f"  Grade only: {df_quality['grade'].corr(df_quality['price']):.4f}")
        print(f"  Condition only: {df_quality['condition'].corr(df_quality['price']):.4f}")
        
        # Visualization
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        sns.scatterplot(data=df_quality, x='quality_score', y='price', ax=axes[0], alpha=0.5)
        axes[0].set_title('Quality Score (Multiplicative) vs. Price')
        axes[0].set_xlabel('Grade × Condition')
        
        sns.scatterplot(data=df_quality, x='quality_grade_condition', y='price', ax=axes[1], alpha=0.5)
        axes[1].set_title('Quality Score (Additive) vs. Price')
        axes[1].set_xlabel('Grade + Condition')
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Missing 'grade' or 'condition' columns.")

## 7. Log Transformations

### Why?
- **Skewed Data:** Price, sqft are right-skewed (few expensive/large properties).
- **Model Assumption:** Linear models assume normally distributed features.
- **Reduces Outlier Impact:** Compresses the upper tail.
- **Interpretability:** Log(price) changes → % changes in price.

### When to Use:
- Features with right skew (most values low, some very high)
- When you care about % changes, not absolute changes
- For regression where predictions shouldn't be negative

### Example:
- **Price:** $100K → $200K (2x) vs. $500K → $1M (2x)
  - Absolute: difference is $100K vs. $500K (5x different)
  - Log: log(2) vs. log(2) (same % change)
  - Log-transformation makes the model more fair

### Formula:
```
log_feature = log1p(feature)  # log1p = log(1 + x) to handle zeros
```

In [ ]:
df_log = df.copy()

# Apply log transforms to right-skewed features
log_features = ['price', 'sqft_living', 'sqft_lot']

for feature in log_features:
    if feature in df_log.columns:
        df_log[f'{feature}_log'] = np.log1p(df_log[feature])
        print(f"✅ Created log_{feature}")

# Comparison visualization
if 'price' in df_log.columns:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Price distribution
    axes[0, 0].hist(df_log['price'], bins=50, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Original Price Distribution (Right-Skewed)')
    axes[0, 0].set_ylabel('Count')
    
    # Log-price distribution
    axes[0, 1].hist(df_log['price_log'], bins=50, color='lightgreen', edgecolor='black')
    axes[0, 1].set_title('Log-Price Distribution (More Normal)')
    axes[0, 1].set_ylabel('Count')
    
    # Q-Q plots
    from scipy import stats
    stats.probplot(df_log['price'], dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title('Q-Q Plot: Original Price')
    
    stats.probplot(df_log['price_log'], dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title('Q-Q Plot: Log-Price (Better Fit)')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("\nSkewness Comparison:")
    from scipy.stats import skew
    print(f"Original Price Skewness: {skew(df_log['price'].dropna()):.3f}")
    print(f"Log-Price Skewness: {skew(df_log['price_log'].dropna()):.3f}")
    print("\n(Skewness closer to 0 is better for linear models)")

## 8. Days on Market

### Why?
- **Market Health:** Quick sales (few days) → high demand/good price; slow sales → may indicate overpricing.
- **Pricing Signal:** Properties that sit long often sell below asking.
- **Buyer Behavior:** Days on market affects final price negotiation.

### Interpretation:
- **1-7 days:** Hot market, strong demand
- **8-30 days:** Normal market
- **30+ days:** Potential issues (overpriced, poor condition, location)

### Note:
In this pipeline, we estimate based on sale date (assuming ~30 day listing period). Ideally, you'd have actual listing dates.

In [ ]:
df_days = df.copy()

# Create days_on_market feature (estimated)
if 'year' in df_days.columns and 'month' in df_days.columns:
    # Estimate with 30-day average
    df_days['days_on_market'] = 30  # Could be more sophisticated
    print("✅ Days on Market Feature Created (estimated at 30 days)")
    
    # If you have actual listing dates, use:
    # sale_date = pd.to_datetime(df_days[['year', 'month']].assign(day=15))
    # listing_date = ... # your actual listing date
    # df_days['days_on_market'] = (sale_date - listing_date).dt.days
    
    if 'price' in df_days.columns:
        print(f"\nAverage Days on Market: {df_days['days_on_market'].mean():.1f}")
        print("\n💡 Tip: If you have actual listing_date and sale_date:")
        print("   days_on_market = (sale_date - listing_date).days")
else:
    print("⚠️ Missing date components.")

## 9. Feature Scaling

### Why Scale?
- **Model Fairness:** Features on same scale contribute equally.
- **Distance-based Models:** KNN, K-Means use distances (need same scale).
- **Gradient Descent:** Faster convergence with scaled features.
- **Interpretability:** Standardized coefficients are comparable.

### Common Methods:
1. **StandardScaler:** (x - mean) / std → mean=0, std=1
2. **MinMaxScaler:** (x - min) / (max - min) → [0, 1]
3. **RobustScaler:** Uses percentiles (robust to outliers)

### When NOT to Scale:
- Tree-based models (XGBoost, RandomForest) - order matters, not magnitude
- Linear models with interpretable coefficients - you may want original scale

In [ ]:
df_scale = df.copy()

# Select numeric columns
numeric_cols = df_scale.select_dtypes(include=[np.number]).columns.tolist()
exclude_cols = ['id', 'price']  # Don't scale ID or target
scale_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"Columns to scale ({len(scale_cols)}): {scale_cols[:5]}...")

# Apply StandardScaler
scaler = StandardScaler()
df_scale[scale_cols] = scaler.fit_transform(df_scale[scale_cols])

print("✅ Features Scaled with StandardScaler")

# Comparison
print("\nBefore Scaling:")
print(f"  sqft_living: mean={df['sqft_living'].mean():.0f}, std={df['sqft_living'].std():.0f}")
print(f"  bedrooms: mean={df['bedrooms'].mean():.2f}, std={df['bedrooms'].std():.2f}")

print("\nAfter Scaling:")
print(f"  sqft_living: mean={df_scale['sqft_living'].mean():.2f}, std={df_scale['sqft_living'].std():.2f}")
print(f"  bedrooms: mean={df_scale['bedrooms'].mean():.2f}, std={df_scale['bedrooms'].std():.2f}")

print("\n✓ Both features now have mean≈0 and std≈1")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if 'sqft_living' in df.columns:
    axes[0].hist(df['sqft_living'], bins=50, alpha=0.7, label='Original', color='blue')
    axes[0].set_title('Original sqft_living Distribution')
    axes[0].set_xlabel('Square Feet')
    
if 'sqft_living' in df_scale.columns:
    axes[1].hist(df_scale['sqft_living'], bins=50, alpha=0.7, label='Scaled', color='green')
    axes[1].set_title('Scaled sqft_living Distribution')
    axes[1].set_xlabel('Standardized Value')

plt.tight_layout()
plt.show()

## 10. Complete Feature Engineering Pipeline

Let's apply all transformations together and save the processed data:

In [ ]:
class FeatureEngineeringPipeline:
    """Complete feature engineering pipeline"""
    
    def __init__(self, df):
        self.df = df.copy()
        self.original_shape = df.shape
    
    def run(self):
        """Execute all transformations"""
        print("Starting Feature Engineering Pipeline")
        print("="*60)
        
        self.temporal_features()
        self.renovation_features()
        self.property_age()
        self.space_ratios()
        self.quality_scores()
        self.log_transforms()
        self.scale_features()
        
        print("\n" + "="*60)
        print(f"Original shape: {self.original_shape}")
        print(f"Final shape: {self.df.shape}")
        print(f"Features added: {self.df.shape[1] - self.original_shape[1]}")
        
        return self.df
    
    def temporal_features(self):
        """Create temporal features"""
        if 'date' in self.df.columns:
            self.df['date'] = pd.to_datetime(self.df['date'], format='%Y%m%dT%H%M%S')
            self.df['year'] = self.df['date'].dt.year
            self.df['month'] = self.df['date'].dt.month
            self.df['season'] = self.df['month'].apply(self._get_season)
            self.df.drop('date', axis=1, inplace=True)
            print("✅ Temporal features")
    
    def renovation_features(self):
        """Create renovation features"""
        if 'yr_renovated' in self.df.columns:
            self.df['was_renovated'] = (self.df['yr_renovated'] > 0).astype(int)
            if 'year' in self.df.columns:
                self.df['years_since_renovation'] = self.df['year'] - self.df['yr_renovated']
                self.df.loc[self.df['yr_renovated'] == 0, 'years_since_renovation'] = 0
            print("✅ Renovation features")
    
    def property_age(self):
        """Create property age feature"""
        if 'yr_built' in self.df.columns and 'year' in self.df.columns:
            self.df['property_age'] = self.df['year'] - self.df['yr_built']
            print("✅ Property age")
    
    def space_ratios(self):
        """Create space ratio features"""
        features = []
        if 'sqft_living' in self.df.columns and 'sqft_lot' in self.df.columns:
            self.df['sqft_per_lot'] = self.df['sqft_living'] / (self.df['sqft_lot'] + 1)
            features.append('sqft_per_lot')
        if 'bedrooms' in self.df.columns and 'bathrooms' in self.df.columns:
            self.df['total_rooms'] = self.df['bedrooms'] + self.df['bathrooms']
            self.df['bedrooms_per_bathroom'] = self.df['bedrooms'] / (self.df['bathrooms'] + 0.01)
            features.extend(['total_rooms', 'bedrooms_per_bathroom'])
        if 'sqft_living' in self.df.columns and 'sqft_living15' in self.df.columns:
            self.df['above_neighborhood'] = (self.df['sqft_living'] > self.df['sqft_living15']).astype(int)
            features.append('above_neighborhood')
        if features:
            print(f"✅ Space ratios: {', '.join(features)}")
    
    def quality_scores(self):
        """Create quality interaction features"""
        if 'grade' in self.df.columns and 'condition' in self.df.columns:
            self.df['quality_score'] = self.df['grade'] * self.df['condition']
            self.df['quality_grade_condition'] = self.df['grade'] + self.df['condition']
            print("✅ Quality scores")
    
    def log_transforms(self):
        """Apply log transformations"""
        log_features = ['price', 'sqft_living', 'sqft_lot']
        for feature in log_features:
            if feature in self.df.columns:
                self.df[f'{feature}_log'] = np.log1p(self.df[feature])
        print(f"✅ Log transforms")
    
    def scale_features(self):
        """Scale numeric features"""
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        exclude_cols = ['id', 'price']
        scale_cols = [col for col in numeric_cols if col not in exclude_cols]
        
        scaler = StandardScaler()
        self.df[scale_cols] = scaler.fit_transform(self.df[scale_cols])
        print(f"✅ Scaled {len(scale_cols)} features")
    
    @staticmethod
    def _get_season(month):
        if month in [12, 1, 2]:
            return 0  # Winter
        elif month in [3, 4, 5]:
            return 1  # Spring
        elif month in [6, 7, 8]:
            return 2  # Summer
        else:
            return 3  # Fall

# Execute pipeline
pipeline = FeatureEngineeringPipeline(df)
df_engineered = pipeline.run()

print("\n" + "="*60)
print("Sample of engineered features:")
display(df_engineered.head())

## 11. Feature Importance Check

Let's verify which engineered features are most correlated with price:

In [ ]:
if 'price' in df_engineered.columns:
    # Calculate correlations
    numeric_features = df_engineered.select_dtypes(include=[np.number]).columns
    correlations = df_engineered[numeric_features].corr()['price'].sort_values(ascending=False)
    
    print("Top 15 Features Correlated with Price:")
    print("="*50)
    display(correlations.head(15))
    
    # Visualization
    plt.figure(figsize=(10, 8))
    correlations.head(15).plot(kind='barh', color='steelblue')
    plt.title('Top 15 Features Correlated with Price')
    plt.xlabel('Correlation Coefficient')
    plt.tight_layout()
    plt.show()

## 12. Summary & Best Practices


| Feature Type | Purpose | Example | When to Use |
|---|---|---|---|
| **Temporal** | Capture market trends | year, season | Time series data |
| **Ratio** | Efficiency & proportions | sqft_per_lot | Domain relationships |
| **Interaction** | Non-linear relationships | grade × condition | Combined effects matter |
| **Log Transform** | Normalize skewed data | log(price) | Right-skewed distributions |
| **Binary** | Yes/no indicators | was_renovated | Categorical importance |
| **Scaling** | Standardization | (x - mean)/std | Distance-based models |



### Next Steps:
1. Save engineered features to CSV
2. Train baseline model (without engineered features)
3. Train model with engineered features
4. Compare performance metrics (RMSE, R²)
5. Drop features that hurt performance (curse of dimensionality)

In [ ]:
# Save engineered features
output_path = '../data/processed/train_engineered.csv'
df_engineered.to_csv(output_path, index=False)
print(f"✅ Engineered features saved to: {output_path}")
print(f"\nFinal dataset shape: {df_engineered.shape}")